In [7]:
# Data manipulation
import os
import pandas as pd
import csv
import json
import jsonlines
import jsonlines as jl
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from dotenv import load_dotenv
from typing import List
import re

# Machine Learning
import torch
import asyncio
import aiohttp
import torch.nn as nn
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Transformers and Langchain
from pydantic import ValidationError, BaseModel, Field

# API and utility
from huggingface_hub import login
from together import Together
import time
from tqdm import tqdm
import accelerate

load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('TOGETHER_API_KEY')
client = Together(api_key=api_key)

In [8]:
class RelationClassification(BaseModel):
    id: str = Field(description="The ID of the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (1: attacks, 3: no use, 5: supports)") 

In [9]:
def classify_text(id: str, comment_text: str, argument: str) -> dict:
    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment about gay marriage in relation to a gay marriage. You need to:
            Identify if the comment makes use of the given argument. Assign the following labels:
            - 1 if the comment attacks the argument explicitly.
            - 2 if the comment attacks the argument implicitly/vaguely.
            - 3 if the comment makes no use of the argument.
            - 4 if the comment supports the argument implicitly/vaguely.
            - 5 if the comment supports the argument explicitly.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0,
        top_p=1,
        top_k=1,
        response_format={
            "type": "json_object",
            "schema": RelationClassification.model_json_schema(),
        }
    )
    return json.loads(extract.choices[0].message.content)

In [10]:
gm = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/GM_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str):
    with jsonlines.open(output_file, mode='w') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']
            print(comment_id, comment_text, argument_text)
            try:
                classification = classify_text(comment_id, comment_text, argument_text)
                writer.write(classification)
                print(classification)
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                continue

process_comments_with_arguments(gm, 'comarg_gm_relation_identification_5_llama.jsonl')

Processing comments:   0%|          | 0/1285 [00:00<?, ?it/s]

1arg2 I am pro because I believe, not only should heterosexual couples have the right to get married, but so should homosexual couples. this is partially a biast opinion, but it is an opinion none the less. One day, I would like to be able to marry my girlfriend. It's my right as a human being to marry whoever I want to as long as there is consent. To tell me otherwise is to take away my 14th amendment right. 'Equal protection of the law. Homosexuality is found in over 1500 species. Yet, homophobia is only found in 1. Tell me, which is the one that's not natural now? Gay couples should be able to take advantage of the fiscal and legal benefits of marriage


Processing comments:   0%|          | 1/1285 [00:00<17:36,  1.22it/s]

{'id': '1arg2', 'label': 5}
1arg3 I am pro because I believe, not only should heterosexual couples have the right to get married, but so should homosexual couples. this is partially a biast opinion, but it is an opinion none the less. One day, I would like to be able to marry my girlfriend. It's my right as a human being to marry whoever I want to as long as there is consent. To tell me otherwise is to take away my 14th amendment right. 'Equal protection of the law. Homosexuality is found in over 1500 species. Yet, homophobia is only found in 1. Tell me, which is the one that's not natural now? Gay marriage undermines the institution of marriage, leading to an increase in out of wedlock births and divorce rates


Processing comments:   0%|          | 2/1285 [00:01<16:44,  1.28it/s]

...

{'id': '198arg6', 'label': 1}
198arg7 yeh i have a comment, the bible says that homosexuals are a abomination. now a bunch of f**s want me to disagree with god. aint gonna happen. be all the f** you want to be, but i refuse to go against god almighty, and tell him that his rules are wrong because a bunch of f**s want to satisfy there emoral sexual urges. god has left his rules to live by. and your breaking them. Marriage should be between a man and a woman


Processing comments: 100%|█████████▉| 1284/1285 [28:04<00:01,  1.12s/it]

{'id': '198arg7', 'label': 1}
1arg1 I am pro because I believe, not only should heterosexual couples have the right to get married, but so should homosexual couples. this is partially a biast opinion, but it is an opinion none the less. One day, I would like to be able to marry my girlfriend. It's my right as a human being to marry whoever I want to as long as there is consent. To tell me otherwise is to take away my 14th amendment right. 'Equal protection of the law. Homosexuality is found in over 1500 species. Yet, homophobia is only found in 1. Tell me, which is the one that's not natural now? Gay couples can declare their union without resort to marriage


Processing comments: 100%|██████████| 1285/1285 [28:06<00:00,  1.31s/it]

{'id': '1arg1', 'label': 5}


In [11]:
def classify_text_ugip(id: str, comment_text: str, argument: str) -> dict:
    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about whether "Under God" should appear in the US Pledge of Allegiance. You need to:
            Identify if the comment makes use of the given argument. Assign the following labels:
            - Label 1: if the comment attacks the argument.
            - Label 3: if the comment makes no use of the argument.
            - Label 5: if the comment supports the argument.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0,
        top_p=1,
        top_k=1,
        response_format={
            "type": "json_object",
            "schema": RelationClassification.model_json_schema(),
        }
    )
    return json.loads(extract.choices[0].message.content)

In [12]:
ugip = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/UGIP_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str):
    with jsonlines.open(output_file, mode='w') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']
            try:
                classification = classify_text_ugip(comment_id, comment_text, argument_text)
                writer.write(classification)
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                continue

process_comments_with_arguments(ugip, 'comarg_ugip_relation_identification_5_llama.jsonl')

Processing comments: 100%|██████████| 1013/1013 [22:29<00:00,  1.33s/it]
